In [3]:
import os
import csv
import zipfile
from pathlib import Path

In [12]:
import shutil

class InvoiceProcessor:
    def __init__(self, input_zip, output_zip):
        self.input_zip = input_zip
        self.output_zip = output_zip
        self.extract_dir = "extracted_files"
        self.csv_dir = os.path.join(self.extract_dir, "csv_files")
        self.pdf_dir = os.path.join(self.extract_dir, "pdf_files")
        Path(self.extract_dir).mkdir(exist_ok=True)
        Path(self.csv_dir).mkdir(exist_ok=True)
        Path(self.pdf_dir).mkdir(exist_ok=True)
        self.records = []

    def extract_zip(self):
        with zipfile.ZipFile(self.input_zip, 'r') as zip_ref:
            for file_info in zip_ref.infolist():
                if file_info.filename.endswith('.csv'):
                    zip_ref.extract(file_info, self.csv_dir)
                elif file_info.filename.endswith('.pdf'):
                    zip_ref.extract(file_info, self.pdf_dir)
        print(f"Extracted '{self.input_zip}' to '{self.extract_dir}'")

    def parse_bank_statements(self):
        for root, _, files in os.walk(self.csv_dir):
            for file in files:
                if file.endswith('.csv'):
                    csv_file = os.path.join(root, file)
                    with open(csv_file, mode='r') as f:
                        reader = csv.reader(f)
                        next(reader)  # Skip header row
                        for row in reader:
                            date, merchant, amount = row  # Adjust indices based on your CSV structure
                            self.records.append({
                                'date': date,
                                'merchant': merchant,
                                'amount': amount,
                                'file_name': f"{date}_{merchant}_{amount}.pdf".replace('/', '-')
                            })

    def rename_invoices(self):
        for root, _, files in os.walk(self.pdf_dir):
            for file in files:
                for record in self.records:
                    if record['amount'] in file:
                        old_path = os.path.join(root, file)
                        new_path = os.path.join(root, record['file_name'])
                        os.rename(old_path, new_path)
                        print(f"Renamed '{old_path}' to '{new_path}'")

    def create_zip(self):
        with zipfile.ZipFile(self.output_zip, 'w') as zip_ref:
            for root, _, files in os.walk(self.pdf_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, self.pdf_dir)
                    zip_ref.write(file_path, arcname)
        print(f"Created '{self.output_zip}' from '{self.pdf_dir}'")

    def process_files(self):
        self.extract_zip()
        self.parse_bank_statements()
        self.rename_invoices()
        self.create_zip()


In [15]:
if __name__ == "__main__":
    input_zip = "March.zip"  # Replace with the actual input zip file path
    output_zip = "output.zip"  # Replace with the desired output zip file path

    # Create an instance of FileProcessor
    processor = InvoiceProcessor(input_zip, output_zip)

    # Extract the zip file
    processor.extract_zip()

    # Parse the CSV files to get renaming information
    processor.parse_bank_statements()

    # Rename the invoices based on the parsed CSV data
    processor.rename_invoices()

    # Create a new zip file with the renamed invoices
    processor.create_zip()


FileNotFoundError: [Errno 2] No such file or directory: 'March.zip'